# Machine learning với `H2O`

## Giới thiệu

Tương tự như R, `H2O` phát triển module máy học trên `python`. Từ `h2o` version `3.26.0.7`, H2O cho phép upload ngược lại `MOJO object` để có thể sử dụng trong mô hình.

```python
pip install h2o
```

In [2]:
import h2o

In [4]:
h2o.init(nthreads = -1, max_mem_size = 4)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 24.0.2+12-54, mixed mode, sharing)
  Starting server from C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Admin\AppData\Local\Temp\tmpz_iytu4m
  JVM stdout: C:\Users\Admin\AppData\Local\Temp\tmpz_iytu4m\h2o_Admin_started_from_python.out
  JVM stderr: C:\Users\Admin\AppData\Local\Temp\tmpz_iytu4m\h2o_Admin_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Bangkok
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,4 months and 4 days
H2O_cluster_name:,H2O_from_python_Admin_zhb71h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.984 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [5]:
airlines_data = h2o.import_file("./99_dataset/allyears2k.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
airlines_data.head()

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
1987,10,14,3,741,730,912,849,PS,1451,NA,91,79,nan,23,11,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,15,4,729,730,903,849,PS,1451,NA,94,79,nan,14,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO
1987,10,17,6,741,730,918,849,PS,1451,NA,97,79,nan,29,11,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,18,7,729,730,847,849,PS,1451,NA,78,79,nan,-2,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,NO,NO
1987,10,19,1,749,730,922,849,PS,1451,NA,93,79,nan,33,19,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,21,3,728,730,848,849,PS,1451,NA,80,79,nan,-1,-2,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,NO,NO
1987,10,22,4,728,730,852,849,PS,1451,NA,84,79,nan,3,-2,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO
1987,10,23,5,731,730,902,849,PS,1451,NA,91,79,nan,13,1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,24,6,744,730,908,849,PS,1451,NA,84,79,nan,19,14,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,YES
1987,10,25,7,729,730,851,849,PS,1451,NA,82,79,nan,2,-1,SAN,SFO,447,nan,nan,0,NA,0,nan,nan,nan,nan,nan,YES,NO


In [7]:
## Create a GBM model, only to later export it as a MOJO
from h2o.estimators import H2OGradientBoostingEstimator
original_model = H2OGradientBoostingEstimator(ntrees = 1)
original_model.train(x = ["Origin", "Dest"], y = "IsDepDelayed", training_frame=airlines_data)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_model_python_1753980869731_1


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    1                  1                           1050                   5            5            5             32            32            32

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.2447292682717408
RMSE: 0.4947011908937968
LogLoss: 0.6825726339390924
Mean Per-Class Error: 0.4195995045566052
AUC: 0.674679575109205
AUCPR: 0.6903867273197087
Gini: 0.3493591502184099

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5084441442563068
       NO    YES    Error    Rate
-----  ----  -----  -------  -----------------
NO     5336  15551  0.7445   (15551.0/20887.0)
YES    2186  20905  0.0947   (2186.0/23091.0)
Total  7522  36456  0.4033   (17737.0/43978.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.508444     0.702134  24
max f2                       0.49727      0.847521  30
max f0point5                 0.524712     0.63887   15
max accuracy                 0.523089     0.623607  17
max precision                0.569162     0.969231  0
max recall                   0.484046     1         31
max specificity              0.569162     0.999904  0
max absolute_mcc             0.523089     0.24412   17
max min_per_class_accuracy   0.5253       0.600191  14
max mean_per_class_accuracy  0.5253       0.617661  14
max tns                      0.569162     20885     0
max fns                      0.569162     23028     0
max fps                      0.484046     20887     31
max tps                      0.484046     23091     31
max tnr                      0.569162     0.999904  0
max fnr                      0.569162     0.997272  0
max fpr                      0.484046     1         31
max tpr                      0.484046     1         31

Gains/Lift Table: Avg response rate: 52.51 %, avg score: 52.50 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain       cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  ---------  -----------------  --------------------
1        0.0156214                   0.563484           1.74653   1.74653            0.917031         0.564021  0.917031                    0.564021            0.0272834       0.0272834                  74.6532    74.6532            0.0245544
2        0.0369503                   0.550921           1.50252   1.60568            0.788913         0.551348  0.843077                    0.556706            0.0320471       0.0593305                  50.2525    60.5683            0.0471219
3        0.0505253                   0.549358           1.4643    1.5677             0.768844         0.549358  0.823132                    0.554732            0.0198779       0.0792083                  46.4303    56.7698            0.0603928
4        0.153213                    0.547794           1.43432   1.4783             0.7531           0.547794  0.776195                    0.550082            0.147287        0.226495                   43.4318    47.8303            0.154297
5        0.275683                    0.535261           1.24188   1.37328            0.652061         0.537736  0.721049                    0.544597

In [8]:
original_model.accuracy

<bound method H2OBinomialModel.accuracy of H2OGradientBoostingEstimator({'parms': {'model_id': {'__meta': {'schema_version': 3, 'schema_name': 'ModelParameterSchemaV3', 'schema_type': 'Iced'}, 'name': 'model_id', 'label': 'model_id', 'help': 'Destination id for this model; auto-generated if not specified.', 'required': False, 'type': 'Key<Model>', 'default_value': None, 'actual_value': {'__meta': {'schema_version': 3, 'schema_name': 'ModelKeyV3', 'schema_type': 'Key<Model>'}, 'name': 'GBM_model_python_1753980869731_1', 'type': 'Key<Model>', 'URL': '/3/Models/GBM_model_python_1753980869731_1'}, 'input_value': None, 'level': 'critical', 'values': [], 'is_member_of_frames': [], 'is_mutually_exclusive_with': [], 'gridable': False}, 'training_frame': {'__meta': {'schema_version': 3, 'schema_name': 'ModelParameterSchemaV3', 'schema_type': 'Iced'}, 'name': 'training_frame', 'label': 'training_frame', 'help': 'Id of the training data frame.', 'required': False, 'type': 'Key<Frame>', 'default_v

In [9]:
import pandas as pd

In [10]:
# Convert sang H2o Frame
test_data = pd.DataFrame({'Origin': ['SAN'], 'Dest': ['SFO']})
test_data_h2o = h2o.H2OFrame(test_data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
# Dự báo
original_model.predict(test_data_h2o)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict,NO,YES
YES,0.475288,0.524712


**Cách download file MOJO**

In [12]:
original_model_filename = original_model.download_mojo(path="../datasets/model/", get_genmodel_jar=True)

In [13]:
original_model_filename

'd:\\01-github\\ds-book-python\\datasets\\model\\GBM_model_python_1753980869731_1.zip'

In [14]:
# Upload mojo back
mojo_model = h2o.upload_mojo(original_model_filename)

generic Model Build progress: |██████████████████████████████████████████████████| (done) 100%


In [15]:
result = mojo_model.predict(airlines_data.head())

generic prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [16]:
# Convert từ H2oFrame sang DataFrame
type(h2o.as_list(result))

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


pandas.core.frame.DataFrame

**Lưu ý**:

- Mô hình `MOJO` có thể load ngược lại vào R và sử dụng cho production

## Tài liệu tham khảo 

- http://docs.h2o.ai/h2o-tutorials/latest-stable/